### 문제 정의
- 백포도주의 열 가지 서로 다른 화학적 특성을 고려해 백포도주 데이터에 존재하는 알코올의 비율 예측 

#### Error 무시

In [11]:
import warnings

warnings.filterwarnings(action='ignore') 

### 라이브러리 import 

In [51]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf 

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model 

from sklearn.metrics import mean_absolute_error

### 경로 설정 

In [15]:
TRAIN_DATA = './data/train/train_data.csv'
VAL_DATA = './data/val/val_data.csv'
TEST_DATA = './data/test/test_data.csv'

In [29]:
train = pd.read_csv(TRAIN_DATA)

train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,5.6,0.46,0.24,4.8,0.042,24.0,72.0,0.99080,3.29,0.37,12.6,6
1,6.5,0.18,0.41,14.2,0.039,47.0,129.0,0.99678,3.28,0.72,10.3,7
2,6.0,0.26,0.32,3.5,0.028,29.0,113.0,0.99120,3.40,0.71,12.3,7
3,7.4,0.18,0.40,1.6,0.047,22.0,102.0,0.99370,3.28,0.44,10.7,5
4,7.6,0.18,0.28,7.1,0.041,29.0,110.0,0.99652,3.20,0.42,9.2,6


In [28]:
train

[0, 1]

### 함수 정의

In [45]:
# 데이터 로드 함수 
def load_data():
    train = pd.read_csv(TRAIN_DATA)
    val = pd.read_csv(VAL_DATA)
    test = pd.read_csv(TEST_DATA)
    
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    val = scaler.fit_transform(val)
    test = scaler.fit_transform(test)
    
    data = dict()
    data['train_y'] = train[:, 10]
    data['train_x'] = train[:, 0:10]
    data['val_y'] = val[:, 10]
    data['val_x'] = val[:, 0:10]
    data['test_y'] = test[:, 10]
    data['test_x'] = test[:, 0:10] 
    
    # scaler를 유지함으로써 예측을 다시 원래 크기로 복원할 수 있도록 함 
    data['scaler'] = scaler 
    
    return data 

In [47]:
# 모델 생성 함수 
def build_network(input_features=10):
    inputs = Input(shape=(input_features, ), name='input')
    x = Dense(32, activation='relu', name='hidden')(inputs)
    prediction = Dense(1, activation='linear', name='final')(x)
    
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

In [46]:
data = load_data()

In [48]:
model = build_network(input_features=10)

### 모델 학습

In [49]:
model.fit(x=data['train_x'], y=data['train_y'], batch_size=32, epochs=200, verbose=1, validation_data=(data['val_x'], data['val_y']))

Train on 3918 samples, validate on 490 samples
Epoch 1/200
3918/3918 [==============================] - 0s 92us/sample - loss: 0.5781 - val_loss: 0.4348
Epoch 2/200
3918/3918 [==============================] - 0s 41us/sample - loss: 0.4005 - val_loss: 0.3204
Epoch 3/200
3918/3918 [==============================] - 0s 41us/sample - loss: 0.3100 - val_loss: 0.2622
Epoch 4/200
3918/3918 [==============================] - 0s 45us/sample - loss: 0.2616 - val_loss: 0.2387
Epoch 5/200
3918/3918 [==============================] - 0s 41us/sample - loss: 0.2382 - val_loss: 0.2292
Epoch 6/200
3918/3918 [==============================] - 0s 44us/sample - loss: 0.2290 - val_loss: 0.2281
Epoch 7/200
3918/3918 [==============================] - 0s 41us/sample - loss: 0.2227 - val_loss: 0.2242
Epoch 8/200
3918/3918 [==============================] - 0s 39us/sample - loss: 0.2186 - val_loss: 0.2281
Epoch 9/200
3918/3918 [==============================] - 0s 39us/sample - loss: 0.2170 - val_loss: 0.2226

Epoch 153/200
3918/3918 [==============================] - 0s 38us/sample - loss: 0.1798 - val_loss: 0.2027
Epoch 154/200
3918/3918 [==============================] - 0s 39us/sample - loss: 0.1802 - val_loss: 0.2039
Epoch 155/200
3918/3918 [==============================] - 0s 39us/sample - loss: 0.1787 - val_loss: 0.2026
Epoch 156/200
3918/3918 [==============================] - 0s 38us/sample - loss: 0.1800 - val_loss: 0.2047
Epoch 157/200
3918/3918 [==============================] - 0s 40us/sample - loss: 0.1807 - val_loss: 0.2063
Epoch 158/200
3918/3918 [==============================] - 0s 38us/sample - loss: 0.1797 - val_loss: 0.2029
Epoch 159/200
3918/3918 [==============================] - 0s 37us/sample - loss: 0.1801 - val_loss: 0.2012
Epoch 160/200
3918/3918 [==============================] - 0s 37us/sample - loss: 0.1791 - val_loss: 0.2051
Epoch 161/200
3918/3918 [==============================] - 0s 36us/sample - loss: 0.1797 - val_loss: 0.2067
Epoch 162/200
3918/3918 [===

### 모델 검증 

In [52]:
print('Model Train MAE:' + str(mean_absolute_error(data['train_y'], model.predict(data['train_x']))))
print('Model Val MAE:' + str(mean_absolute_error(data['val_y'], model.predict(data['val_x']))))
print('Model Test MAE:' + str(mean_absolute_error(data['test_y'], model.predict(data['test_x']))))

Model Train MAE:0.17883858166928981
Model Val MAE:0.2069404551712891
Model Test MAE:0.20101230697665914


### 모델 저장 & 로드 

In [53]:
model.save('regression_model.h5')

In [1]:
from tensorflow.keras.models import load_model

model = load_model('regression_model.h5') 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa